# 3 Pointers Made against game_totals.csv

### Import packages

In [43]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
from scipy.stats import pearsonr
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [44]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [45]:
df = pd.read_csv('backend/data/totals/game_totals.csv').drop(['Unnamed: 0'], axis=1)

### Basic exploration

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40218 entries, 0 to 40217
Data columns (total 23 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     40218 non-null  object 
 1   visitor  40218 non-null  object 
 2   home     40218 non-null  object 
 3   team     40218 non-null  int64  
 4   fg       40202 non-null  float64
 5   fga      40202 non-null  float64
 6   fg_perc  40202 non-null  float64
 7   3p       40202 non-null  float64
 8   3pa      40202 non-null  float64
 9   3p_perc  40202 non-null  float64
 10  ft       40202 non-null  float64
 11  fta      40202 non-null  float64
 12  ft_perc  40202 non-null  float64
 13  orb      40202 non-null  float64
 14  drb      40202 non-null  float64
 15  trb      40202 non-null  float64
 16  ast      40202 non-null  float64
 17  stl      40202 non-null  float64
 18  blk      40202 non-null  float64
 19  tov      40202 non-null  float64
 20  pf       40202 non-null  float64
 21  pts      402

In [47]:
df.head()

,date,visitor,home,team,fg,fga,fg_perc,3p,3pa,3p_perc,ft,fta,ft_perc,orb,drb,trb,ast,stl,blk,tov,pf,pts,starter
0,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,39.0,79.0,0.494,7.0,13.0,0.538,23.0,32.0,0.719,13.0,36.0,49.0,22.0,10.0,5.0,16.0,21.0,108.0,NaN
1,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,1,25.0,65.0,0.385,3.0,17.0,0.176,13.0,22.0,0.591,4.0,25.0,29.0,13.0,6.0,3.0,21.0,24.0,66.0,NaN
2,"Tue, Oct 31, 2006",Phoenix Suns,Los Angeles Lakers,0,40.0,77.0,0.519,13.0,30.0,0.433,13.0,17.0,0.765,4.0,25.0,29.0,29.0,7.0,5.0,21.0,25.0,106.0,NaN
3,"Tue, Oct 31, 2006",Phoenix Suns,Los Angeles Lakers,1,46.0,83.0,0.554,6.0,12.0,0.500,16.0,24.0,0.667,12.0,31.0,43.0,30.0,11.0,1.0,20.0,19.0,114.0,NaN
4,"Wed, Nov 1, 2006",Indiana Pacers,Charlotte Bobcats,0,35.0,86.0,0.407,6.0,16.0,0.375,30.0,41.0,0.732,18.0,32.0,50.0,26.0,14.0,5.0,17.0,26.0,106.0,NaN


In [48]:
# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])


# Conver 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])


# Merge in opponents (see team defensive stats)
df = pd.merge(
    df, 
    df, 
    left_on=['date', 'visitor', 'home'], 
    right_on=['date', 'visitor', 'home'],
    suffixes=('', '_opp'),
    how='left')

df = df[df['team'] != df['team_opp']]


df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40218 entries, 1 to 80434
Data columns (total 43 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         40218 non-null  datetime64[ns]
 1   visitor      40218 non-null  object        
 2   home         40218 non-null  object        
 3   team         40218 non-null  object        
 4   fg           40202 non-null  float64       
 5   fga          40202 non-null  float64       
 6   fg_perc      40202 non-null  float64       
 7   3p           40202 non-null  float64       
 8   3pa          40202 non-null  float64       
 9   3p_perc      40202 non-null  float64       
 10  ft           40202 non-null  float64       
 11  fta          40202 non-null  float64       
 12  ft_perc      40202 non-null  float64       
 13  orb          40202 non-null  float64       
 14  drb          40202 non-null  float64       
 15  trb          40202 non-null  float64       
 16  ast 

# Dataframe of team's last 15 performances

In [49]:
# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = df[df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

df['dates'] = df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
df['date_1'], df['date_2'] = df['dates'].apply(lambda x: x[0]), df['dates'].apply(lambda x: x[1])
df['date_3'], df['date_4'] = df['dates'].apply(lambda x: x[2]), df['dates'].apply(lambda x: x[3])
df['date_5'], df['date_6'] = df['dates'].apply(lambda x: x[4]), df['dates'].apply(lambda x: x[5])
df['date_7'], df['date_8'] = df['dates'].apply(lambda x: x[6]), df['dates'].apply(lambda x: x[7])
df['date_9'], df['date_10'] = df['dates'].apply(lambda x: x[8]), df['dates'].apply(lambda x: x[9])
df['date_11'], df['date_12'] = df['dates'].apply(lambda x: x[10]), df['dates'].apply(lambda x: x[11])
df['date_13'], df['date_14'] = df['dates'].apply(lambda x: x[12]), df['dates'].apply(lambda x: x[13])
df['date_15'] = df['dates'].apply(lambda x: x[14])

In [50]:
# Statistics
stats = ['fg', 'fga', '3p', '3pa', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts',
         'fg_opp', 'fga_opp', '3p_opp', '3pa_opp', 'ft_opp', 'fta_opp', 'orb_opp', 'drb_opp', 'trb_opp', 'ast_opp', 
         'stl_opp', 'blk_opp', 'tov_opp', 'pf_opp', 'pts_opp']

# X and y column names to merge on
x_cols = ['date', 'team'] + stats

last_15_games = df.copy()
last_15_games.loc[:, 'target'] = last_15_games.loc[:, '3p']
X = df[x_cols].copy()

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

In [51]:
# Calculate z-score
def z_score(value, mean, std):
    return (value - mean) / std

### Last 15 Performances (Unweighted)

In [52]:
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))]

last_15 = last_15_games[cols].copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15[stat] = 0
    for date in dates:
        last_15[stat] = last_15[stat] + last_15[stat + date]
    
    last_15[stat] = last_15[stat] / len(dates)
    
# Calculate standard deviation for each stat over a team's performance
for stat in stats:
    last_15[stat + '_std'] = 0
    for date in dates:
        last_15[stat + '_std'] = last_15[stat + '_std'] + ((last_15[stat + date] - last_15[stat]) ** 2)
    
    last_15[stat + '_std'] = last_15[stat + '_std'] / len(dates)
    last_15[stat + '_std'] = last_15[stat + '_std'] ** .5

# Feature engineer trends
for stat in stats:
    last_15[stat + '_trend'] = 0
    for date in dates[:3]:
        last_15[stat + '_trend'] = last_15[stat + '_trend'] + \
                                        z_score(last_15[stat + date], last_15[stat], last_15[stat + '_std']).fillna(0)
    
    last_15[stat + '_trend'] = last_15[stat + '_trend'] / len(dates[:3])

last_15 = last_15.groupby(['date', 'visitor', 'home']).sum()

# Standard deviation and trending cols
std_cols = [stat + '_std' for stat in stats]
trend_cols = [stat + '_trend' for stat in stats]

# Keep columns
last_15 = last_15[['target'] + stats + std_cols + trend_cols].dropna(axis=0)

# Calculate percentages
last_15['fg_perc'] = last_15['fg'] / last_15['fga']
last_15['fg_perc_opp'] = last_15['fg_opp'] / last_15['fga_opp']

last_15['3p_perc'] = last_15['3p'] / last_15['3pa']
last_15['3p_perc_opp'] = last_15['3p_opp'] / last_15['3pa_opp']

last_15['ft_perc'] = last_15['ft'] / last_15['fta']
last_15['ft_perc_opp'] = last_15['ft_opp'] / last_15['fta_opp']

# Calculate advanced stats
last_15['ts_perc'] = last_15['pts'] / (2 * (last_15['fga'] + .44 * last_15['fta']))
last_15['ts_perc_opp'] = last_15['pts_opp'] / (2 * (last_15['fga_opp'] + .44 * last_15['fta_opp']))

last_15['efg_perc'] = (last_15['fg'] + (.5 * last_15['3p'])) / last_15['fga']
last_15['efg_perc_opp'] = (last_15['fg_opp'] + (.5 * last_15['3p_opp'])) / last_15['fga_opp']

last_15['3par'] = last_15['3pa'] / last_15['fga']
last_15['3par_opp'] = last_15['3pa_opp'] / last_15['fga_opp']

last_15['ftr'] = last_15['fta'] / last_15['fga']
last_15['ftr_opp'] = last_15['fta_opp'] / last_15['fga_opp']

last_15['orb_perc'] = last_15['orb'] / (last_15['orb'] + last_15['drb_opp'])
last_15['orb_perc_opp'] = last_15['orb_opp'] / (last_15['orb_opp'] + last_15['drb'])

last_15['drb_perc'] = last_15['drb'] / (last_15['drb'] + last_15['orb_opp'])
last_15['drb_perc_opp'] = last_15['drb_opp'] / (last_15['drb_opp'] + last_15['orb'])

last_15['trb_perc'] = last_15['trb'] / (last_15['trb'] + last_15['trb_opp'])
last_15['trb_perc_opp'] = last_15['trb_opp'] / (last_15['trb_opp'] + last_15['trb'])

last_15['ast_perc'] = last_15['ast'] / last_15['fg']
last_15['ast_perc_opp'] = last_15['ast_opp'] / last_15['fg_opp']
        
last_15 = last_15.dropna(axis=0)
last_15.tail(5)

target         fg  \
date       visitor              home                                       
2022-03-06 New Orleans Pelicans Denver Nuggets            0.0  84.600000   
           New York Knicks      Los Angeles Clippers      0.0  81.066667   
           Phoenix Suns         Milwaukee Bucks           0.0  87.266667   
           Toronto Raptors      Cleveland Cavaliers       0.0  79.933333   
           Utah Jazz            Oklahoma City Thunder     0.0  80.466667   

                                                              fga         3p  \
date       visitor              home                                           
2022-03-06 New Orleans Pelicans Denver Nuggets         171.800000  24.200000   
           New York Knicks      Los Angeles Clippers   177.466667  25.200000   
           Phoenix Suns         Milwaukee Bucks        177.066667  24.800000   
           Toronto Raptors      Cleveland Cavaliers    172.600000  22.933333   
           Utah Jazz            Oklahoma City Thunder  176.000000  25.600000   

                                                             3pa         ft  \
date       visitor              home                                          
2022-03-06 New Orleans Pelicans Denver Nuggets         69.333333  36.200000   
           New York Knicks      Los Angeles Clippers   69.866667  34.600000   
           Phoenix Suns         Milwaukee Bucks        69.200000  36.933333   
           Toronto Raptors      Cleveland Cavaliers    62.066667  34.466667   
           Utah Jazz            Oklahoma City Thunder  74.600000  32.066667   

                                                             fta        orb  \
date       visitor              home                                          
2022-03-06 New Orleans Pelicans Denver Nuggets         44.600000  19.933333   
           New York Knicks      Los Angeles Clippers   46.866667  22.000000   
           Phoenix Suns         Milwaukee Bucks        47.533333  18.333333   
           Toronto Raptors      Cleveland Cavaliers    45.800000  23.400000   
           Utah Jazz            Oklahoma City Thunder  41.733333  21.666667   

                                                             drb        trb  \
date       visitor              home                                          
2022-03-06 New Orleans Pelicans Denver Nuggets         71.000000  90.933333   
           New York Knicks      Los Angeles Clippers   70.933333  92.933333   
           Phoenix Suns         Milwaukee Bucks        71.133333  89.466667   
           Toronto Raptors      Cleveland Cavaliers    63.600000  87.000000   
           Utah Jazz            Oklahoma City Thunder  69.733333  91.400000   

                                                             ast        stl  \
date       visitor              home                                          
2022-03-06 New Orleans Pelicans Denver Nuggets         53.666667  15.600000   
           New York Knicks      Los Angeles Clippers   47.800000  14.533333   
           Phoenix Suns         Milwaukee Bucks        54.000000  15.333333   
           Toronto Raptors      Cleveland Cavaliers    46.666667  15.000000   
           Utah Jazz            Oklahoma City Thunder  46.533333  16.866667   

                                                             blk        tov  \
date       visitor              home                                          
2022-03-06 New Orleans Pelicans Denver Nuggets          8.000000  28.400000   
           New York Knicks      Los Angeles Clippers   10.466667  26.666667   
           Phoenix Suns         Milwaukee Bucks         7.666667  23.266667   
           Toronto Raptors      Cleveland Cavaliers     8.066667  25.466667   
           Utah Jazz            Oklahoma City Thunder   9.133333  28.200000   

                                                              pf         pts  \
date       visitor              home                                           
2022-03-06 New Orleans Pelicans

## Correlations

In [53]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in last_15:
    corr_p = pearsonr(last_15['target'], last_15[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print correlations
corr_df = corr_df[corr_df['p-value'] < .05].drop(['p-value'], axis=1).sort_values(['corr'], axis=0, ascending=False)
corr_df

,corr,stat
0,1.00,target
4,0.74,3pa
101,0.74,3par
3,0.73,3p
19,0.69,3pa_opp
...,...,...
7,-0.29,orb
103,-0.39,ftr
104,-0.42,ftr_opp
106,-0.46,orb_perc_opp


## Save dataframe with significantly correlated stats

In [63]:
stats = corr_df[corr_df['corr'].abs() >= .6]['stat']
df = last_15[stats]

df.to_csv('backend/data/inputs/3p/game_totals.csv')